> Proszę przedstawić W jaki sposób zrealizować strukturę danych, która pozwala wykonywać operacje:

1. Insert

2. RemoveMedian (wyciągnięcie mediany)

###### tak, żeby wszystkie operacje działały w czasie $ O(log(n)) $.

### Plan działania

Naszym celem jest podział zbioru wartości, z którego mamy wyznaczyć medianę, na 2 podzbiory, z których jeden będzie zawierał wartości mniejsze od mediany, a drugi większe od mediany (mediana może należeć do jednego z nich - wyjaśnienie później). Na myśl nasuwa się posortowanie tablicy, ale nie o to chodzi w tym zadaniu, bo musimy zapewnić, że wstawianie do struktury zajmie czas $ O(log(n)) $ oraz odczyt mediany również $ O(log(n)) $. Przedstawione wymagania, dotyczące pożądanej złożoności obliczeniowej, sugerują nam od razu zastosowanie struktury, dla której operacja wstawiania i usuwania wartości ma złożoność $ O(log(n)) $, a więc skorzystanie z kopca binarnego. Ponieważ planujemy podział wartości na 2 (prawie) równe (mogą się różnić o 1 element) podzbiory, potrzebne nam będą 2 kopce. Najłatwiej osiągniemy pożądany efekt, budując nową strukturę w taki sposób, by korzystała zarówno z kopca Max Heap, jak i Min Heap.

Przejdźmy teraz do omówienia, jaka jest logika dzialania tego algorytmu. Ponieważ Max Heap umieszcza wartość największą w swoim korzeniu, a Min Heap wartość najmniejszą, Max Heap musi zawierać tę połowę (lub blisko połowę) wartości, które są mniejsze od mediany (oraz ewentualnie również medianę), a Min Heap - wartości od niej większe (lub również medianę, jeżeli nie ma jej w Max Heapie).

Tworzenie struktury zajmie czas $ O(n \cdot log(n)) $. Postępujemy tak, że przeprowadzamy iterację przez sekwencję wartości, podejmując decyzję, do którego kopca "wrzucić" bieżącą wartość. Ponieważ założyliśmy, że kopiec Min Heap ma wartości większe, jeżeli bieżąca wartość jest większa od wartości najmniejszej w Min Heapie, musimy bieżącą wartość wrzucić do Min Heapa, natomiast, w przeciwnym wypadku, wartość wrzucamy do Max Heapa (ponieważ jest mniejsza od najmniejszej wartości spośród wartości największych (około połowy wartości)). Po każdym takim wstawieniu konieczne jest sprawdzenie, czy nasza struktura jest nadal zbalansowana (czy różnica między liczbą elementów w Min Heapie a liczbą elementów w Max Heapie jest nie większa niż 1). Jeżeli okaże się, że jest niezbalansowana, musimy z większego obecnie kopca zabrać roota i przerzucić do drugiego kopca (oczywiście taka operacja wymaga zbalansowania obu kopców, dlatego najlepiej posłużyć się wcześniej poznaną implementacją strkuktury kopca).

Poza zaimplementowaniem metod/funkcji, zwracających medianę i dodających wartość do struktrury, zaimplementowałem również usuwanie mediany (nie jest to częścią zadania).

##### Funkcja pomocnicza formatująca kompletne drzewo binarne

In [1]:
def complete_tree_string(values):
    if values:
        just = 0
        data = []

        limit = 1
        values_row = []
        branches_row = []
        prev_nodes = 0

        for i in range(1, len(values) + 1):
            curr_nodes = i - prev_nodes
            val_str = str(values[i-1])
            just = max(just, len(val_str))
            values_row.append(val_str)
            right_child_idx = 2 * i
            left_child_idx = right_child_idx - 1
            if left_child_idx < len(values):
                branches_row.append('/')
            if right_child_idx < len(values):
                branches_row.append('\\')

            if curr_nodes == limit: 
                prev_nodes = i
                limit *= 2
                data.append([values_row, branches_row])
                values_row = []
                branches_row = []

        if values_row:
            data.append([values_row, branches_row])

        begin_sep = sep = 3 if just % 2 else 2
        data_iter = iter(data[::-1])
        result = [''] * (len(data) * 2 - 1)
        result[-1] = (' ' * sep).join(val.center(just) for val in next(data_iter)[0])

        # Format the tree string
        for i, (values, branches) in enumerate(data_iter):
            mul = 2 * i + 1
            # Values
            indent = (2 ** (i + 1) - 1) * (just + begin_sep) // 2
            sep = 2 * sep + just
            result[-(mul + 2)] = f"{' ' * indent}{(' ' * sep).join(val.center(just) for val in values)}"
            # Branches
            branch_indent = (3 * indent + just) // 4
            branches_row = []
            d_indent = indent - branch_indent
            branches_sep = ' ' * (2 * (d_indent - 1) + just)
            for i in range(0, len(branches), 2):
                branches_row.append(f"{branches[i]}{branches_sep}{branches[i + 1] if i + 1 < len(branches) else ''}")
            result[-(mul + 1)] = f"{' ' * branch_indent}{(' ' * (sep - 2 * d_indent)).join(branches_row)}"

        return '\n'.join(result)
    else:
        return ''

### Implementacja algorytmu (obiektowa)

In [2]:
class MaxHeap:
    def __init__(self, values=None):
        self.heap = []
    
    def __bool__(self):
        return bool(self.heap)
    
    @property
    def heap_size(self):
        return len(self.heap)
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, val: object):
        # Add a value as the last node of out Complete Binary Tree
        self.heap.append(val)
        # Fix heap in order to satisfy a max-heap property
        self._heapify_up(self.heap_size - 1)
        
    def get_max(self) -> object:
        return None if not self.heap else self.heap[0]
        
    def remove_max(self) -> object:
        if self.heap_size == 0:
            raise IndexError(f'remove_max from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        last = self.heap.pop()
        if self.heap_size > 0:
            self.heap[0] = last
            # Fix a heap in order to stisfy a max-heap property
            self._heapify_down(0, self.heap_size)
        return removed
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0): 
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] > self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):
        # Loop till the current node has a child larger than itself
        # We assume that when we enter a node which both children are
        # smaller than this node, a subtree which a current node is a
        # root of must fulfill a max-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            largest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] > self.heap[curr_idx]: 
                    largest_idx = l
                if r < end_idx and self.heap[r] > self.heap[largest_idx]:
                    largest_idx = r

            if largest_idx != curr_idx:
                self.swap(curr_idx, largest_idx)
                curr_idx = largest_idx
            else:
                break


class MinHeap:
    def __init__(self):
        self.heap = []
    
    def __bool__(self):
        return bool(self.heap)
    
    @property
    def heap_size(self):
        return len(self.heap)
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, val: object):
        # Add a value as the last node of our Complete Binary Tree
        self.heap.append(val)
        # Fix heap in order to satisfy a min-heap property
        self._heapify_up(self.heap_size - 1)
        
    def get_min(self) -> object:
        return None if not self.heap else self.heap[0]
        
    def remove_min(self) -> object:
        if self.heap_size == 0:
            raise IndexError(f'remove_min from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        last = self.heap.pop()
        if self.heap_size > 0:
            self.heap[0] = last
            # Fix a heap in order to stisfy a min-heap property
            self._heapify_down(0, self.heap_size)
        return removed
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] < self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx): 
        # Loop till the current node has a child smaller than itself
        # We assume that when we enter a node which both children are
        # larger than this node, a subtree which a current node is a
        # root of must fulfill a min-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            smallest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] < self.heap[curr_idx]: 
                    smallest_idx = l
                if r < end_idx and self.heap[r] < self.heap[smallest_idx]:
                    smallest_idx = r

            if smallest_idx != curr_idx:
                self.swap(curr_idx, smallest_idx)
                curr_idx = smallest_idx
            else:
                break

            
class MedianHeap:
    def __init__(self, values=[]):
        self.min_heap = MinHeap()
        self.max_heap = MaxHeap()
        for val in values:
            self.insert(val)
            
    def __str__(self):
        max_str = complete_tree_string(self.max_heap.heap)
        min_str = complete_tree_string(self.min_heap.heap)
        return f"Max Heap:\n{max_str}\n\nMin Heap:\n{min_str}"
        
    def __len__(self):
        return self.max_heap.heap_size + self.min_heap.heap_size
    
    def insert(self, val: object):  # O(log n)
        # If no value is stored in this data structure or a value inserted
        # is greater than the lowest value of the Min Heap, insert this value
        # to the Min Heap
        if not self or val > self.min_heap.get_min():
            self.min_heap.insert(val)
        # Add a value to the Max Heap in other cases
        else:
            self.max_heap.insert(val)
        # Make heaps have the same (differing only by one element) sizes
        self._rebalance_heaps()
            
    def get_median(self):  # O(1)
        # If there are more element in the min_heap, our set of
        # values has odd number of values and median is this heap's root
        if self.min_heap.heap_size < self.max_heap.heap_size:
            return self.max_heap.get_max()
        # If the Max Heap is bigger, its root is a median
        elif self.min_heap.heap_size > self.max_heap.heap_size:
            return self.min_heap.get_min()
        # If we have even number of elements
        elif self:
            res = (self.max_heap.get_max() + self.min_heap.get_min()) / 2
            return int(res) if res == int(res) else res
        # If there is no value stored in this data structure
        else:
            return None
    
    # Note that we don't have to rebalance our structure after removing a median
    # That's because if a structure has odd number of values, the one from the
    # greater heap will be removed, thus both heaps will have the same size at
    # the end. In next removals roots of both heaps will be removed as our 
    # structure has now even number of values stored.
    def remove_median(self):  # O(log n)
        # If there are more element in the min_heap, our set of
        # values has odd number of values and median is this heap's root
        if self.min_heap.heap_size < self.max_heap.heap_size:
            return self.max_heap.remove_max()
        # If the Max Heap is bigger, its root is a median
        elif self.min_heap.heap_size > self.max_heap.heap_size:
            return self.min_heap.remove_min()
        # If we have even number of elements
        elif self:
            res = (self.max_heap.remove_max() + self.min_heap.remove_min()) / 2
            return int(res) if res == int(res) else res
        else:
            raise IndexError(f'remove_median from an empty {self.__class__.__name__}')
        
    def _rebalance_heaps(self):
        d_size = self.min_heap.heap_size - self.max_heap.heap_size 
        # If the Min Heap has greater size by more than one element than
        # the Max Heap, move its root to the Max Heap
        if d_size > 1:
            self.max_heap.insert(self.min_heap.remove_min())
        # On the other hand, id the Max Heap is greater in size by more than
        # one element than the Min Heap, move its root to the Min Heap
        elif d_size < -1:
            self.min_heap.insert(self.max_heap.remove_max())

###### Kilka testów

In [3]:
import random

samples = 100

passed = 0
for n in range(samples):
    arr = [random.randint(0, 100) for _ in range(random.randint(0, 30))]
    arr_sorted = sorted(arr)
    center = len(arr) // 2
    if len(arr) >= 2:
        expected = arr_sorted[center] if len(arr) % 2 else (arr_sorted[center] + arr_sorted[center - 1]) / 2
    elif len(arr) == 1:
        expected = arr[0]
    else:
        expected = None
    median_heap = MedianHeap(arr)
    result = median_heap.get_median()
    is_correct = result == expected
    passed += is_correct
    print(f'TEST #{n + 1}:')
    print('Input:', arr)
    print('Sorted:', sorted(arr))
    print('Length:', len(arr))
    print('Expected', expected)
    print('Result:', result)
    print(f'Total passed to tested ratio: {passed}/{n + 1}')
    print()

TEST #1:
Input: [3, 64, 51, 53, 96, 64, 39, 20, 82, 19, 17, 35, 9, 23, 95, 73]
Sorted: [3, 9, 17, 19, 20, 23, 35, 39, 51, 53, 64, 64, 73, 82, 95, 96]
Length: 16
Expected 45.0
Result: 45
Total passed to tested ratio: 1/1

TEST #2:
Input: [85, 93, 63, 33, 59, 21, 76, 41, 55, 75]
Sorted: [21, 33, 41, 55, 59, 63, 75, 76, 85, 93]
Length: 10
Expected 61.0
Result: 61
Total passed to tested ratio: 2/2

TEST #3:
Input: [49, 57, 45, 61, 91, 79, 63, 60, 86, 77, 65, 27, 23, 100, 11, 19, 37]
Sorted: [11, 19, 23, 27, 37, 45, 49, 57, 60, 61, 63, 65, 77, 79, 86, 91, 100]
Length: 17
Expected 60
Result: 60
Total passed to tested ratio: 3/3

TEST #4:
Input: [55, 7, 18, 58, 20, 72, 58, 11, 54, 20, 37, 87, 50, 25, 20, 98, 18, 49]
Sorted: [7, 11, 18, 18, 20, 20, 20, 25, 37, 49, 50, 54, 55, 58, 58, 72, 87, 98]
Length: 18
Expected 43.0
Result: 43
Total passed to tested ratio: 4/4

TEST #5:
Input: [53, 98, 58]
Sorted: [53, 58, 98]
Length: 3
Expected 58
Result: 58
Total passed to tested ratio: 5/5

TEST #6:
Inp

In [4]:
while median_heap:
    print(median_heap)
    print('\nRemoved median:', median_heap.remove_median(), end='\n\n')

Max Heap:
      31
     /  \
  24      20
  /\
1   22

Min Heap:
      54
     /  \
  65      74
  /\      /
97  72  95

Removed median: 54

Max Heap:
      31
     /  \
  24      20
  /\
1   22

Min Heap:
      65
     /  \
  72      74
  /\
97  95

Removed median: 48

Max Heap:
      24
     /  \
  22      20
  /
1 

Min Heap:
      72
     /  \
  95      74
  /
97

Removed median: 48

Max Heap:
  22
  /\
1   20

Min Heap:
  74
  /\
95  97

Removed median: 48

Max Heap:
  20
  /
1 

Min Heap:
  95
  /
97

Removed median: 57.5

Max Heap:
1

Min Heap:
97

Removed median: 49



### Implementacja algorytmu (funkcyjna)

In [5]:
_left = lambda i: 2 * i + 1
_right = lambda i: 2 * i + 2
_parent = lambda i: (i - 1) // 2


def print_heap(heap: list, *args, **kwargs):
    print(complete_tree_string(heap), *args, **kwargs)
    
    
def _swap(heap: list, i, j):
    heap[i], heap[j] = heap[j], heap[i]
    
"""
Max Heap
"""        
def insert_to_max_heap(heap: list, val: object):
    # Add a value as the last node of out Complete Binary Tree
    heap.append(val)
    # Fix heap in order to satisfy a max-heap property
    _heapify_max_up(heap, len(heap) - 1)
    
    
def remove_max_from_heap(heap: list) -> object:
    if not heap:
        raise IndexError(f'remove_max from an empty Max Heap')
    # Store a value to be returned
    removed = heap[0]
    # Place the last leaf in the root position
    last = heap.pop()
    if heap:
        heap[0] = last
        # Fix a heap in order to stisfy a max-heap property
        _heapify_max_down(heap, 0, len(heap))
    return removed
    
    
def _heapify_max_up(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index' = 0):
    while curr_idx > end_idx:
        parent_idx = _parent(curr_idx)
        if heap[curr_idx] > heap[parent_idx]:
            _swap(heap, curr_idx, parent_idx)
        curr_idx = parent_idx
            
    
def _heapify_max_down(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index'):
    # Loop till the current node has a child larger than itself
    # We assume that when we enter a node which both children are
    # smaller than this node, a subtree which a current node is a
    # root of must fulfill a max-heap property.
    while True:
        i = _left(curr_idx)
        j = _right(curr_idx)
        k = curr_idx
        
        if i < end_idx:
            if heap[i] > heap[k]:
                k = i
            if j < end_idx and heap[j] > heap[k]:
                k = j
                
        if k == curr_idx: return
        # Swap the current with the largest child
        heap[curr_idx], heap[k] = heap[k], heap[curr_idx]
        curr_idx = k

"""
Min Heap
"""         
def insert_to_min_heap(heap: list, val: object):
    # Add a value as the last node of our Complete Binary Tree
    heap.append(val)
    # Fix heap in order to satisfy a min-heap property
    _heapify_min_up(heap, len(heap) - 1)
    
    
def remove_min_from_heap(heap: list) -> object:
    if not heap:
        raise IndexError(f'remove_min from an empty Min Heap')
    # Store a value to be returned
    removed = heap[0]
    # Place the last leaf in the root position
    last = heap.pop()
    if heap:
        heap[0] = last
        # Fix a heap in order to stisfy a min-heap property
        _heapify_min_down(heap, 0, len(heap))
    return removed
    
    
def _heapify_min_up(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index' = 0):
    while curr_idx > end_idx:
        parent_idx = _parent(curr_idx)
        if heap[curr_idx] < heap[parent_idx]:
            _swap(heap, curr_idx, parent_idx)
        curr_idx = parent_idx
            
    
def _heapify_min_down(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index'):
    # Loop till the current node has a child smaller than itself
    # We assume that when we enter a node which both children are
    # larger than this node, a subtree which a current node is a
    # root of must fulfill a min-heap property
    while True:
        i = _left(curr_idx)
        j = _right(curr_idx)
        k = curr_idx
        
        if i < end_idx:
            if heap[i] < heap[k]:
                k = i
            if j < end_idx and heap[j] < heap[k]:
                k = j
                
        if k == curr_idx: return
        # Swap the current with the smallest child
        heap[curr_idx], heap[k] = heap[k], heap[curr_idx]
        curr_idx = k
        
"""
Median Heap
"""   
def create_median_heap(values: 'iterable') -> ('min_heap', 'max_heap'):
    min_heap = []
    max_heap = []
    for val in values:
        insert_to_median_heap(min_heap, max_heap, val)
    return min_heap, max_heap


def print_median_heap(min_heap: list, max_heap: list):
    print("Max Heap")
    print_heap(max_heap, end='\n\n')
    print("Min Heap")
    print_heap(min_heap)
    
    
def insert_to_median_heap(min_heap: list, max_heap: list, val: object):
    if not min_heap or min_heap[0] < val:
        insert_to_min_heap(min_heap, val)
    else:
        insert_to_max_heap(max_heap, val)
    _rebalance_median_heap(min_heap, max_heap)
    

def get_median_from_median_heap(min_heap: list, max_heap: list) -> 'median':
    if len(min_heap) < len(max_heap):
        return max_heap[0]
    elif len(min_heap) > len(max_heap):
        return min_heap[0]
    elif min_heap and max_heap:
        res = (min_heap[0] + max_heap[0]) / 2
        return int(res) if int(res) == res else res
    else:
        return None
           
        
def remove_median_from_median_heap(min_heap: list, max_heap: list) -> 'median':
    if len(min_heap) < len(max_heap):
        return remove_max_from_heap(max_heap)
    elif len(min_heap) > len(max_heap):
        return remove_min_from_heap(min_heap)
    elif min_heap and max_heap:
        res = (remove_max_from_heap(max_heap) + remove_min_from_heap(min_heap)) / 2
        return int(res) if int(res) == res else res
    else:
        raise IndexError(f'remove_median from an empty median heap')

        
def _rebalance_median_heap(min_heap: list, max_heap: list):
    d_size = len(min_heap) - len(max_heap)
    if d_size > 1:
        insert_to_max_heap(max_heap, remove_min_from_heap(min_heap))
    elif d_size < -1:
        insert_to_min_heap(min_heap, remove_max_from_heap(max_heap))

###### Kilka testów

In [6]:
import random

samples = 100

passed = 0
for n in range(samples):
    arr = [random.randint(0, 100) for _ in range(random.randint(0, 30))]
    arr_sorted = sorted(arr)
    center = len(arr) // 2
    if len(arr) >= 2:
        expected = arr_sorted[center] if len(arr) % 2 else (arr_sorted[center] + arr_sorted[center - 1]) / 2
    elif len(arr) == 1:
        expected = arr[0]
    else:
        expected = None
    min_heap, max_heap = create_median_heap(arr)
    result = get_median_from_median_heap(min_heap, max_heap)
    is_correct = result == expected
    passed += is_correct
    print(f'TEST #{n + 1}:')
    print('Input:', arr)
    print('Sorted:', sorted(arr))
    print('Length:', len(arr))
    print('Expected', expected)
    print('Result:', result)
    print(f'Total passed to tested ratio: {passed}/{n + 1}')
    print()

TEST #1:
Input: [88, 83, 58, 99, 50, 98, 81, 72, 96, 61, 4, 75, 10, 84, 67, 95, 34, 84, 39, 20, 35, 0, 71, 22]
Sorted: [0, 4, 10, 20, 22, 34, 35, 39, 50, 58, 61, 67, 71, 72, 75, 81, 83, 84, 84, 88, 95, 96, 98, 99]
Length: 24
Expected 69.0
Result: 69
Total passed to tested ratio: 1/1

TEST #2:
Input: [61, 70, 78, 68, 63, 32, 59, 92, 74, 83, 9, 63, 38, 34, 86]
Sorted: [9, 32, 34, 38, 59, 61, 63, 63, 68, 70, 74, 78, 83, 86, 92]
Length: 15
Expected 63
Result: 63
Total passed to tested ratio: 2/2

TEST #3:
Input: [63, 34, 11, 76, 79, 30, 30, 50, 63, 95, 24, 71, 61, 64, 28, 99, 93, 63]
Sorted: [11, 24, 28, 30, 30, 34, 50, 61, 63, 63, 63, 64, 71, 76, 79, 93, 95, 99]
Length: 18
Expected 63.0
Result: 63
Total passed to tested ratio: 3/3

TEST #4:
Input: [86, 44, 17, 9, 4, 37, 7, 11, 58, 54, 46, 20, 23, 33, 73, 55, 1, 69, 8, 62, 71, 45, 37, 71, 51, 12, 59, 21, 43, 57]
Sorted: [1, 4, 7, 8, 9, 11, 12, 17, 20, 21, 23, 33, 37, 37, 43, 44, 45, 46, 51, 54, 55, 57, 58, 59, 62, 69, 71, 71, 73, 86]
Lengt

In [7]:
while min_heap or max_heap:
    print_median_heap(min_heap, max_heap)
    print('\nRemoved median:', remove_median_from_median_heap(min_heap, max_heap), end='\n\n')

Max Heap
      37
     /  \
  19      36
  /\      /
7   14  10

Min Heap
          63
       /     \
    66          85
   / \         / 
 71    67   100

Removed median: 50

Max Heap
      36
     /  \
  19      10
  /\
7   14

Min Heap
          66
       /     \
    67          85
   / \
 71   100

Removed median: 51

Max Heap
      19
     /  \
  14      10
  /
7 

Min Heap
          67
       /     \
    71          85
   / 
100

Removed median: 43

Max Heap
  14
  /\
7   10

Min Heap
    71
   / \
100    85

Removed median: 42.5

Max Heap
  10
  /
7 

Min Heap
    85
   / 
100

Removed median: 47.5

Max Heap
7

Min Heap
100

Removed median: 53.5

